https://nbviewer.jupyter.org/github/justmarkham/pandas-videos/blob/master/top_25_pandas_tricks.ipynb

# Pandas 활용

In [1]:
import pandas as pd
import numpy as np


## rename columns

In [3]:
df = pd.DataFrame({'col one':[100, 200], 'col two':[300, 400]})
df

,col one,col two
0,100,300
1,200,400


In [5]:
#아래 세가지 모두 동일한 결과를 나타냄
df = df.rename({'col one':'col_one', 'col two':'col_two'}, axis='columns')
df.columns = ['col_one', 'col_two']
df.columns = df.columns.str.replace(' ', '_')
df

,col_one,col_two
0,100,300
1,200,400


In [8]:
#컬럼 앞에 추가
df.add_prefix('X_')

,X_col_one,X_col_two
0,100,300
1,200,400


In [9]:
#컬럼 뒤에 추가
df.add_suffix('_Y')

,col_one_Y,col_two_Y
0,100,300
1,200,400


## reverse row order

In [12]:
drinks = pd.read_csv('http://bit.ly/drinksbycountry')

In [13]:
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa


In [14]:
drinks.loc[::-1].head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
192,Zimbabwe,64,18,4,4.7,Africa
191,Zambia,32,19,4,2.5,Africa
190,Yemen,6,0,0,0.1,Asia
189,Vietnam,111,2,1,2.0,Asia
188,Venezuela,333,100,3,7.7,South America


In [15]:
drinks.loc[::-1].reset_index(drop=True).head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Zimbabwe,64,18,4,4.7,Africa
1,Zambia,32,19,4,2.5,Africa
2,Yemen,6,0,0,0.1,Asia
3,Vietnam,111,2,1,2.0,Asia
4,Venezuela,333,100,3,7.7,South America


## reverse column order

In [16]:
drinks.loc[:, ::-1].head()

,continent,total_litres_of_pure_alcohol,wine_servings,spirit_servings,beer_servings,country
0,Asia,0.0,0,0,0,Afghanistan
1,Europe,4.9,54,132,89,Albania
2,Africa,0.7,14,0,25,Algeria
3,Europe,12.4,312,138,245,Andorra
4,Africa,5.9,45,57,217,Angola


## select columns by data type

In [17]:
drinks.dtypes

country                          object
beer_servings                     int64
spirit_servings                   int64
wine_servings                     int64
total_litres_of_pure_alcohol    float64
continent                        object
dtype: object

In [21]:
drinks.select_dtypes(include='number').head()
#number : int, float 등 포함

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
0,0,0,0,0.0
1,89,132,54,4.9
2,25,0,14,0.7
3,245,138,312,12.4
4,217,57,45,5.9


In [22]:
drinks.select_dtypes(include='object').head()

,country,continent
0,Afghanistan,Asia
1,Albania,Europe
2,Algeria,Africa
3,Andorra,Europe
4,Angola,Africa


In [23]:
drinks.select_dtypes(include=['number', 'object', 'category', 'datetime']).head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa


In [24]:
#number로 되어있는 column은 제외(exclude)
drinks.select_dtypes(exclude='number').head()

,country,continent
0,Afghanistan,Asia
1,Albania,Europe
2,Algeria,Africa
3,Andorra,Europe
4,Angola,Africa


## Convert strings to numbers★★

In [34]:
df = pd.DataFrame({'col_one':['1.1', '2.2', '3.3'],
                   'col_two':['4.4', '5.5', '6.6'],
                   'col_three':['7.7', '8.8', '-']})
df

,col_one,col_two,col_three
0,1.1,4.4,7.7
1,2.2,5.5,8.8
2,3.3,6.6,-


In [35]:
df.dtypes

col_one      object
col_two      object
col_three    object
dtype: object

In [37]:
#아래 두가지 모두 동일한 결과

#df['col_one']=pd.to_numeric(df['col_one'])
df=df.astype({'col_one':'float', 'col_two':'float'}) #three열에는 적용x ('-')
df.dtypes

col_one      float64
col_two      float64
col_three     object
dtype: object

**알아두기 :**
**df=df.astype({'col_one':'float', 'col_two':'float'})**

In [38]:
pd.to_numeric(df.col_three, errors='coerce')

0    7.7
1    8.8
2    NaN
Name: col_three, dtype: float64

In [39]:
pd.to_numeric(df.col_three, errors='coerce').fillna(0)

0    7.7
1    8.8
2    0.0
Name: col_three, dtype: float64

In [40]:
df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
df

,col_one,col_two,col_three
0,1.1,4.4,7.7
1,2.2,5.5,8.8
2,3.3,6.6,0.0


In [60]:
df.dtypes

col_one      float64
col_two      float64
col_three    float64
dtype: object

## Build a DataFrame from multiple files (row-wise)
### 다양한 파일로 부터 하나의 데이터 프레임 만들기

In [61]:
stocks = pd.read_csv('http://bit.ly/smallstocks', parse_dates=['Date'])

In [ ]:
#왜 안되는지 모르겟음 ㅠ 

#pd.read_csv('data/stocks1.csv')
#pd.read_csv('data/stocks2.csv')
#pd.read_csv('data/stocks3.csv')

In [59]:
from glob import glob
stock_files = sorted(glob('data/stocks*.csv')) #stocks~로 시작되는 파일
stock_files
#['data/stocks1.csv', 'data/stocks2.csv', 'data/stocks3.csv']

[]

In [62]:
#pd.concat((pd.read_csv(file) for file in stock_files))
#인덱스가 0,1,2,0,1,2,....로 출력

In [ ]:
#pd.concat((pd.read_csv(file) for file in stock_files), ignore_index=True)
#인덱스가 0,1,2,3,4,....로 출력

## Create a DataFrame from the clipboard
- 내가 클립보드에 복사하고 있는 표?를 데이터 프레임으로 만들 수 있음

In [64]:
df = pd.read_clipboard()
df

,Column A,Column B,Column C
0,1,4.4,seven
1,2,5.5,eight
2,3,6.6,nine


## Split a DataFrame into two random subsets

In [65]:
movies = pd.read_csv('http://bit.ly/imdbratings')

In [66]:
len(movies)

979

In [68]:
movies_1 = movies.sample(frac=0.75, random_state=1234)
#75%를 randomly하게 가져옴

In [69]:
movies_2 = movies.drop(movies_1.index)
#movie1에 있는 것을 drop함으로 써 나머지 25% 가져오기

In [70]:
len(movies_1) + len(movies_2)

979

In [71]:
movies_1.index.sort_values()
#sort_values():기본 오름차순으로 정렬

Int64Index([  0,   2,   5,   6,   7,   8,   9,  11,  13,  16,
            ...
            966, 967, 969, 971, 972, 974, 975, 976, 977, 978],
           dtype='int64', length=734)

In [72]:
movies_2.index.sort_values()
#index가 유니크 해야만 이와 같이 나타날 수 있음 주의

Int64Index([  1,   3,   4,  10,  12,  14,  15,  18,  26,  30,
            ...
            931, 934, 937, 941, 950, 954, 960, 968, 970, 973],
           dtype='int64', length=245)

## Filter a DataFrame by multiple categories
- isin 활용하기
- ~ 사용

In [78]:
movies.head()

,star_rating,title,content_rating,genre,duration,actors_list
0,9.3,The Shawshank Redemption,R,Crime,142,"[u'Tim Robbins', u'Morgan Freeman', u'Bob Gunt..."
1,9.2,The Godfather,R,Crime,175,"[u'Marlon Brando', u'Al Pacino', u'James Caan']"
2,9.1,The Godfather: Part II,R,Crime,200,"[u'Al Pacino', u'Robert De Niro', u'Robert Duv..."
3,9.0,The Dark Knight,PG-13,Action,152,"[u'Christian Bale', u'Heath Ledger', u'Aaron E..."
4,8.9,Pulp Fiction,R,Crime,154,"[u'John Travolta', u'Uma Thurman', u'Samuel L...."


In [79]:
movies.genre.unique()

array(['Crime', 'Action', 'Drama', 'Western', 'Adventure', 'Biography',
       'Comedy', 'Animation', 'Mystery', 'Horror', 'Film-Noir', 'Sci-Fi',
       'History', 'Thriller', 'Family', 'Fantasy'], dtype=object)

In [80]:
movies[(movies.genre == 'Action') |
       (movies.genre == 'Drama') |
       (movies.genre == 'Western')].head()

,star_rating,title,content_rating,genre,duration,actors_list
3,9.0,The Dark Knight,PG-13,Action,152,"[u'Christian Bale', u'Heath Ledger', u'Aaron E..."
5,8.9,12 Angry Men,NOT RATED,Drama,96,"[u'Henry Fonda', u'Lee J. Cobb', u'Martin Bals..."
6,8.9,"The Good, the Bad and the Ugly",NOT RATED,Western,161,"[u'Clint Eastwood', u'Eli Wallach', u'Lee Van ..."
9,8.9,Fight Club,R,Drama,139,"[u'Brad Pitt', u'Edward Norton', u'Helena Bonh..."
11,8.8,Inception,PG-13,Action,148,"[u'Leonardo DiCaprio', u'Joseph Gordon-Levitt'..."


In [81]:
#isin
movies[movies.genre.isin(['Action', 'Drama', 'Western'])].head()

,star_rating,title,content_rating,genre,duration,actors_list
3,9.0,The Dark Knight,PG-13,Action,152,"[u'Christian Bale', u'Heath Ledger', u'Aaron E..."
5,8.9,12 Angry Men,NOT RATED,Drama,96,"[u'Henry Fonda', u'Lee J. Cobb', u'Martin Bals..."
6,8.9,"The Good, the Bad and the Ugly",NOT RATED,Western,161,"[u'Clint Eastwood', u'Eli Wallach', u'Lee Van ..."
9,8.9,Fight Club,R,Drama,139,"[u'Brad Pitt', u'Edward Norton', u'Helena Bonh..."
11,8.8,Inception,PG-13,Action,148,"[u'Leonardo DiCaprio', u'Joseph Gordon-Levitt'..."


In [82]:
#~를 사용함으로써 제외한 것들 출력
movies[~movies.genre.isin(['Action', 'Drama', 'Western'])].head()

,star_rating,title,content_rating,genre,duration,actors_list
0,9.3,The Shawshank Redemption,R,Crime,142,"[u'Tim Robbins', u'Morgan Freeman', u'Bob Gunt..."
1,9.2,The Godfather,R,Crime,175,"[u'Marlon Brando', u'Al Pacino', u'James Caan']"
2,9.1,The Godfather: Part II,R,Crime,200,"[u'Al Pacino', u'Robert De Niro', u'Robert Duv..."
4,8.9,Pulp Fiction,R,Crime,154,"[u'John Travolta', u'Uma Thurman', u'Samuel L...."
7,8.9,The Lord of the Rings: The Return of the King,PG-13,Adventure,201,"[u'Elijah Wood', u'Viggo Mortensen', u'Ian McK..."


## Filter a DataFrame by largest categories
- value_counts()
- nlargest(n)

In [84]:
#value_counts() : count해주기
counts = movies.genre.value_counts()
counts

Drama        278
Comedy       156
Action       136
Crime        124
Biography     77
Adventure     75
Animation     62
Horror        29
Mystery       16
Western        9
Sci-Fi         5
Thriller       5
Film-Noir      3
Family         2
History        1
Fantasy        1
Name: genre, dtype: int64

In [85]:
counts.nlargest(3)

Drama     278
Comedy    156
Action    136
Name: genre, dtype: int64

In [86]:
#isin에 사용하기 위해서는 index object가 필요
counts.nlargest(3).index

Index(['Drama', 'Comedy', 'Action'], dtype='object')

In [87]:
#isin에 리스트 형태가 들어가야함
movies[movies.genre.isin(counts.nlargest(3).index)].head()

,star_rating,title,content_rating,genre,duration,actors_list
3,9.0,The Dark Knight,PG-13,Action,152,"[u'Christian Bale', u'Heath Ledger', u'Aaron E..."
5,8.9,12 Angry Men,NOT RATED,Drama,96,"[u'Henry Fonda', u'Lee J. Cobb', u'Martin Bals..."
9,8.9,Fight Club,R,Drama,139,"[u'Brad Pitt', u'Edward Norton', u'Helena Bonh..."
11,8.8,Inception,PG-13,Action,148,"[u'Leonardo DiCaprio', u'Joseph Gordon-Levitt'..."
12,8.8,Star Wars: Episode V - The Empire Strikes Back,PG,Action,124,"[u'Mark Hamill', u'Harrison Ford', u'Carrie Fi..."


## Handle missing values
- isna()
- dropna()

In [89]:
ufo = pd.read_csv('http://bit.ly/uforeports', parse_dates=['Time'])
ufo.head()

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,1930-06-01 22:00:00
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00
2,Holyoke,NaN,OVAL,CO,1931-02-15 14:00:00
3,Abilene,NaN,DISK,KS,1931-06-01 13:00:00
4,New York Worlds Fair,NaN,LIGHT,NY,1933-04-18 19:00:00


In [90]:
#isna() : true or false로 생성되고 true들만 sum함 =>true가결측치
ufo.isna().sum()

City                  25
Colors Reported    15359
Shape Reported      2644
State                  0
Time                   0
dtype: int64

In [91]:
ufo.isna().mean()

City               0.001371
Colors Reported    0.842004
Shape Reported     0.144948
State              0.000000
Time               0.000000
dtype: float64

In [92]:
#결측치가 있는 columns drop
ufo.dropna(axis='columns').head()

,State,Time
0,NY,1930-06-01 22:00:00
1,NJ,1930-06-30 20:00:00
2,CO,1931-02-15 14:00:00
3,KS,1931-06-01 13:00:00
4,NY,1933-04-18 19:00:00


In [95]:
#len(ufo): total number of rows
#len(ufo) * 0.9= 10%이상의 결측치를 가진 column들을 drop
#thresh : int, optional/ Require that many non-NA values.
ufo.dropna(thresh=len(ufo)*0.9, axis='columns').head()

,City,State,Time
0,Ithaca,NY,1930-06-01 22:00:00
1,Willingboro,NJ,1930-06-30 20:00:00
2,Holyoke,CO,1931-02-15 14:00:00
3,Abilene,KS,1931-06-01 13:00:00
4,New York Worlds Fair,NY,1933-04-18 19:00:00


## Split a string into multiple columns

In [96]:
df = pd.DataFrame({'name':['John Arthur Doe', 'Jane Ann Smith'],
                   'location':['Los Angeles, CA', 'Washington, DC']})
df

,name,location
0,John Arthur Doe,"Los Angeles, CA"
1,Jane Ann Smith,"Washington, DC"


In [99]:
#' ' 공백을 기준으로 split 
# expand=true 하게 되면   return DataFrame/MultiIndex expanding dimensionality.
# ``False``, return Series/Index, containing lists of strings.
df.name.str.split(' ', expand=True) 

,0,1,2
0,John,Arthur,Doe
1,Jane,Ann,Smith


In [100]:
df[['first', 'middle', 'last']] = df.name.str.split(' ', expand=True)
df

,name,location,first,middle,last
0,John Arthur Doe,"Los Angeles, CA",John,Arthur,Doe
1,Jane Ann Smith,"Washington, DC",Jane,Ann,Smith


In [101]:
df.location.str.split(', ', expand=True)

,0,1
0,Los Angeles,CA
1,Washington,DC


In [102]:
df['city'] = df.location.str.split(', ', expand=True)[0]
df

,name,location,first,middle,last,city
0,John Arthur Doe,"Los Angeles, CA",John,Arthur,Doe,Los Angeles
1,Jane Ann Smith,"Washington, DC",Jane,Ann,Smith,Washington


## Expand a Series of lists into a DataFrame

In [103]:
df = pd.DataFrame({'col_one':['a', 'b', 'c'], 'col_two':[[10, 40], [20, 50], [30, 60]]})
df

,col_one,col_two
0,a,"[10, 40]"
1,b,"[20, 50]"
2,c,"[30, 60]"


In [105]:
#col_two는 리스트 형태 
df_new = df.col_two.apply(pd.Series)
df_new

,0,1
0,10,40
1,20,50
2,30,60


In [111]:
pd.concat([df, df_new], axis='columns')

,col_one,col_two,0,1
0,a,"[10, 40]",10,40
1,b,"[20, 50]",20,50
2,c,"[30, 60]",30,60


##  Aggregate by multiple functions
- groupby()
- agg()
- sum()

In [114]:
orders = pd.read_csv('http://bit.ly/chiporders', sep='\t')

In [115]:
orders.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


In [136]:
#orders['item_price'].replace('$','',regex=True)
#$ 지우기 왜 안되지

In [ ]:
orders[orders.order_id == 1].item_price.sum()
#11.56

In [ ]:
orders.groupby('order_id').item_price.sum().head()
#order_id에 따른 각 주문의 합 구하기

In [ ]:
#agg() 함수를 통해 multiple function이 가능
orders.groupby('order_id').item_price.agg(['sum', 'count']).head()

## Combine the output of an aggregation with a DataFrame

In [138]:
orders.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


In [139]:
orders.groupby('order_id').item_price.sum().head()
# $ 해결 후 다시 해보기

order_id
1    $2.39 $3.39 $3.39 $2.39 
2                     $16.98 
3               $10.98 $1.69 
4               $11.75 $9.25 
5                $9.25 $4.45 
Name: item_price, dtype: object

## Select a slice of rows and columns

In [140]:
titanic = pd.read_csv('http://bit.ly/kaggletrain')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [141]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [142]:
titanic.describe().loc['min':'max']

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
min,1.0,0.0,1.0,0.420,0.0,0.0,0.0000
25%,223.5,0.0,2.0,20.125,0.0,0.0,7.9104
50%,446.0,0.0,3.0,28.000,0.0,0.0,14.4542
75%,668.5,1.0,3.0,38.000,1.0,0.0,31.0000
max,891.0,1.0,3.0,80.000,8.0,6.0,512.3292


In [143]:
titanic.describe().loc['min':'max', 'Pclass':'Parch']

,Pclass,Age,SibSp,Parch
min,1.0,0.420,0.0,0.0
25%,2.0,20.125,0.0,0.0
50%,3.0,28.000,0.0,0.0
75%,3.0,38.000,1.0,0.0
max,3.0,80.000,8.0,6.0


## Reshape a MultiIndexed Series
- unstack()

In [148]:
titanic.Survived.mean()

0.3838383838383838

In [149]:
titanic.groupby('Sex').Survived.mean()

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

In [150]:
titanic.groupby(['Sex', 'Pclass']).Survived.mean()

Sex     Pclass
female  1         0.968085
        2         0.921053
        3         0.500000
male    1         0.368852
        2         0.157407
        3         0.135447
Name: Survived, dtype: float64

In [151]:
#unstack() : dataframe 형태로 만들기
titanic.groupby(['Sex', 'Pclass']).Survived.mean().unstack()

Pclass,1,2,3
Sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


## Create a pivot table

In [153]:
titanic.pivot_table(index='Sex', columns='Pclass', values='Survived', aggfunc='mean')

Pclass,1,2,3
Sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [154]:
#margins 을 통해 각 row, column에 따른 total을 추가할 수 있음
titanic.pivot_table(index='Sex', columns='Pclass', values='Survived', aggfunc='mean',
                    margins=True)

Pclass,1,2,3,All
Sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838


In [155]:
titanic.pivot_table(index='Sex', columns='Pclass', values='Survived', aggfunc='count',
                    margins=True)

Pclass,1,2,3,All
Sex,,,,
female,94,76,144,314
male,122,108,347,577
All,216,184,491,891


## Convert continuous data into categorical data
- category
- pd.cut()

In [159]:
titanic.Age.head(10)

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [160]:
#child는 0~18, young adult는 18~25,,,
#이렇게 할 경우 type이 category로 됨을 알 수 있음
pd.cut(titanic.Age, bins=[0, 18, 25, 99], labels=['child', 'young adult', 'adult']).head(10)

0    young adult
1          adult
2          adult
3          adult
4          adult
5            NaN
6          adult
7          child
8          adult
9          child
Name: Age, dtype: category
Categories (3, object): ['child' < 'young adult' < 'adult']

## Change display options(소수점 자리)
- set_option()
- pd.reset_option

In [162]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [163]:
# 소수점 자리
pd.set_option('display.float_format', '{:.2f}'.format)

In [164]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S


In [166]:
#원래대로 돌려놓기
pd.reset_option('display.float_format')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Style a DataFrame
- style.format(highlight_min,max,background_gradient, bar...)


In [171]:
stocks

,Date,Close,Volume,Symbol
0,2016-10-03,31.50,14070500,CSCO
1,2016-10-03,112.52,21701800,AAPL
2,2016-10-03,57.42,19189500,MSFT
3,2016-10-04,113.00,29736800,AAPL
4,2016-10-04,57.24,20085900,MSFT
5,2016-10-04,31.35,18460400,CSCO
6,2016-10-05,57.64,16726400,MSFT
7,2016-10-05,31.59,11808600,CSCO
8,2016-10-05,113.05,21453100,AAPL


In [172]:
format_dict = {'Date':'{:%m/%d/%y}', 'Close':'${:.2f}', 'Volume':'{:,}'}

In [173]:
stocks.style.format(format_dict)

,Date,Close,Volume,Symbol
0,10/03/16,$31.50,"14,070,500",CSCO
1,10/03/16,$112.52,"21,701,800",AAPL
2,10/03/16,$57.42,"19,189,500",MSFT
3,10/04/16,$113.00,"29,736,800",AAPL
4,10/04/16,$57.24,"20,085,900",MSFT
5,10/04/16,$31.35,"18,460,400",CSCO
6,10/05/16,$57.64,"16,726,400",MSFT
7,10/05/16,$31.59,"11,808,600",CSCO
8,10/05/16,$113.05,"21,453,100",AAPL


In [174]:
(stocks.style.format(format_dict)
 .hide_index()
 .highlight_min('Close', color='red')
 .highlight_max('Close', color='lightgreen')
)

Date,Close,Volume,Symbol
10/03/16,$31.50,"14,070,500",CSCO
10/03/16,$112.52,"21,701,800",AAPL
10/03/16,$57.42,"19,189,500",MSFT
10/04/16,$113.00,"29,736,800",AAPL
10/04/16,$57.24,"20,085,900",MSFT
10/04/16,$31.35,"18,460,400",CSCO
10/05/16,$57.64,"16,726,400",MSFT
10/05/16,$31.59,"11,808,600",CSCO
10/05/16,$113.05,"21,453,100",AAPL


In [175]:
(stocks.style.format(format_dict)
 .hide_index()
 .background_gradient(subset='Volume', cmap='Blues')
)

Date,Close,Volume,Symbol
10/03/16,$31.50,"14,070,500",CSCO
10/03/16,$112.52,"21,701,800",AAPL
10/03/16,$57.42,"19,189,500",MSFT
10/04/16,$113.00,"29,736,800",AAPL
10/04/16,$57.24,"20,085,900",MSFT
10/04/16,$31.35,"18,460,400",CSCO
10/05/16,$57.64,"16,726,400",MSFT
10/05/16,$31.59,"11,808,600",CSCO
10/05/16,$113.05,"21,453,100",AAPL


In [176]:
(stocks.style.format(format_dict)
 .hide_index()
 .bar('Volume', color='lightblue', align='zero')
 .set_caption('Stock Prices from October 2016')
)

Date,Close,Volume,Symbol
10/03/16,$31.50,"14,070,500",CSCO
10/03/16,$112.52,"21,701,800",AAPL
10/03/16,$57.42,"19,189,500",MSFT
10/04/16,$113.00,"29,736,800",AAPL
10/04/16,$57.24,"20,085,900",MSFT
10/04/16,$31.35,"18,460,400",CSCO
10/05/16,$57.64,"16,726,400",MSFT
10/05/16,$31.59,"11,808,600",CSCO
10/05/16,$113.05,"21,453,100",AAPL
